# Prerequisites
Install Theano and Lasagne using the following commands:

```bash
pip install -r https://raw.githubusercontent.com/Lasagne/Lasagne/master/requirements.txt
pip install https://github.com/Lasagne/Lasagne/archive/master.zip
```

Working in a virtual environment is recommended.

# Data preparation

Current code allows to generate geodesic patches from a collection of shapes represented as triangular meshes.
To get started with the pre-processing:
```
git clone https://github.com/jonathanmasci/ShapeNet_data_preparation_toolbox.git
```

The usual processing pipeline is show in ```run_forrest_run.m```. 
We will soon update this preparation stage, so perhaps better to start with our pre-computed dataset, and stay tuned! :-)

## Prepared data

All it is required to train on the FAUST_registration dataset for this demo is available for download at
https://www.dropbox.com/s/aamd98nynkvbcop/EG16_tutorial.tar.bz2?dl=0

# ICNN Toolbox

```bash
git clone https://github.com/sosiris/TAU16_seminar.git
```

![](http://www.people.usi.ch/mascij/EG16_tutorial/shapenet_architecture.png)

In [1]:
import sys
import os
import numpy as np
import scipy.io
import time

import theano
import theano.tensor as T
import theano.sparse as Tsp

import lasagne as L
import lasagne.layers as LL
import lasagne.objectives as LO
import lasagne.nonlinearities as LN
from lasagne.layers.normalization import batch_norm

sys.path.append('..')
from icnn import utils_lasagne, dataset, snapshotter

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1080 (CNMeM is enabled with initial size: 70.0% of memory, cuDNN 5105)


## Data loading

In [2]:
reload(dataset)
base_path = './dataset/FAUST_registrations/data/diam=200/'

ds = dataset.ClassificationDatasetPatchesMinimal(
    'FAUST_registrations_train.txt', 'FAUST_registrations_test.txt',
    os.path.join(base_path, 'descs', 'shot'),
    os.path.join(base_path, 'patch_aniso', 'alpha=100_nangles=016_ntvals=005_tmin=6.000_tmax=24.000_thresh=99.900_norm=L1'), 
    None, 
    os.path.join(base_path, 'lbo'),
    os.path.join(base_path, 'labels'),
    epoch_size=50)

Loading train descs
elapsed time 0.632000
Loading test descs
elapsed time 0.562000
Loading train patches
elapsed time 2.359000
Loading test patches
elapsed time 2.310000
Loading train LB bases
elapsed time 0.649000
Loading test LB bases
elapsed time 0.472000
Loading train labels
elapsed time 0.057000
Loading test labels
elapsed time 0.060000


## Network definition

In [5]:
reload(utils_lasagne)
nin = 544
nclasses = 6890
l2_weight = 1e-5
ref_lbo = T.constant(ds.train_lbo[0])

def get_model(inp, patch_op, lb_op):
    print(LL.get_output_shape(inp))
    icnn = batch_norm(utils_lasagne.GCNNLayer([inp, patch_op], 16, nrings=5, nrays=16))
    print(LL.get_output_shape(icnn))
    icnn = batch_norm(utils_lasagne.GCNNLayer([icnn, patch_op], 32, nrings=5, nrays=16))
    print(LL.get_output_shape(icnn))
    icnn = batch_norm(utils_lasagne.GCNNLayer([icnn, patch_op], 20, nrings=5, nrays=16))
    print(LL.get_output_shape(icnn))
    # TODO: add functional mapping computation and application (maybe instead of the dense layer?)
    ffn = utils_lasagne.FMAPLayer([icnn, lb_op], ref_lbo=ref_lbo, neigen=20)
    ffn =  batch_norm(LL.DenseLayer(ffn, 128, nonlinearity=LN.rectify))
    print(LL.get_output_shape(ffn))
    ffn = LL.DenseLayer(ffn, nclasses, nonlinearity=utils_lasagne.log_softmax)

    print(LL.get_output_shape(ffn))
    return ffn

inp = LL.InputLayer(shape=(None, nin))
patch_op = LL.InputLayer(input_var=Tsp.csc_fmatrix('patch_op'), shape=(None, None))
lb_op = LL.InputLayer(input_var=T.matrix('lb_op'), shape=(None, None))

ffn = get_model(inp, patch_op, lb_op)

# L.layers.get_output -> theano variable representing network
output = LL.get_output(ffn)
print(LL.get_output_shape(ffn))
pred = LL.get_output(ffn, deterministic=True)  # in case we use dropout

# target theano variable indicatind the index a vertex should be mapped to wrt the latent space
target = T.ivector('idxs')

# to work with logit predictions, better behaved numerically
cla = utils_lasagne.categorical_crossentropy_logdomain(output, target, nclasses).mean()
acc = LO.categorical_accuracy(pred, target).mean()

# a bit of regularization is commonly used
regL2 = L.regularization.regularize_network_params(ffn, L.regularization.l2)


cost = cla + l2_weight * regL2

(None, 16)
(None, 32)
(None, 64)
(None, 128)
(None, 6890)
(None, 6890)


## Define the update rule, how to train

In [6]:
params = LL.get_all_params(ffn, trainable=True)
grads = T.grad(cost, params)
# computes the L2 norm of the gradient to better inspect training
grads_norm = T.nlinalg.norm(T.concatenate([g.flatten() for g in grads]), 2)

# Adam turned out to be a very good choice for correspondence
updates = L.updates.adam(grads, params, learning_rate=0.001)

## Compile

In [7]:
funcs = dict()
funcs['train'] = theano.function([inp.input_var, patch_op.input_var, lb_op.input_var, target],
                                 [cost, cla, l2_weight * regL2, grads_norm, acc], updates=updates,
                                 on_unused_input='warn')
funcs['acc_loss'] = theano.function([inp.input_var, patch_op.input_var, lb_op.input_var, target],
                                    [acc, cost], on_unused_input='warn')
funcs['predict'] = theano.function([inp.input_var, patch_op.input_var, lb_op.input_var],
                                   [pred], on_unused_input='warn')

# Training (a bit simplified)

In [ ]:
n_epochs = 50
eval_freq = 1

start_time = time.time()
best_trn = 1e5
best_tst = 1e5

kvs = snapshotter.Snapshotter('demo_training.snap')

for it_count in xrange(n_epochs):
    tic = time.time()
    b_l, b_c, b_s, b_r, b_g, b_a = [], [], [], [], [], []
    for x_ in ds.train_iter():
        tmp = funcs['train'](*x_)

        # do some book keeping (store stuff for training curves etc)
        b_l.append(tmp[0])
        b_c.append(tmp[1])
        b_r.append(tmp[2])
        b_g.append(tmp[3])
        b_a.append(tmp[4])
    epoch_cost = np.asarray([np.mean(b_l), np.mean(b_c), np.mean(b_r), np.mean(b_g), np.mean(b_a)])
    print(('[Epoch %03i][trn] cost %9.6f (cla %6.4f, reg %6.4f), |grad| = %.06f, acc = %7.5f %% (%.2fsec)') %
                 (it_count, epoch_cost[0], epoch_cost[1], epoch_cost[2], epoch_cost[3], epoch_cost[4] * 100, 
                  time.time() - tic))

    if np.isnan(epoch_cost[0]):
        print("NaN in the loss function...let's stop here")
        break

    if (it_count % eval_freq) == 0:
        v_c, v_a = [], []
        for x_ in ds.test_iter():
            tmp = funcs['acc_loss'](*x_)
            v_a.append(tmp[0])
            v_c.append(tmp[1])
        test_cost = [np.mean(v_c), np.mean(v_a)]
        print(('           [tst] cost %9.6f, acc = %7.5f %%') % (test_cost[0], test_cost[1] * 100))

        if epoch_cost[0] < best_trn:
            kvs.store('best_train_params', [it_count, LL.get_all_param_values(ffn)])
            best_trn = epoch_cost[0]
        if test_cost[0] < best_tst:
            kvs.store('best_test_params', [it_count, LL.get_all_param_values(ffn)])
            best_tst = test_cost[0]
print("...done training %f" % (time.time() - start_time))

[Epoch 000][trn] cost  8.719976 (cla 8.7119, reg 0.0080), |grad| = 6.197805, acc = 0.01974 % (254.63sec)
           [tst] cost  9.228011, acc = 0.01451 %


ERROR:root:[KVP] Overwriting group best_train_params!
ERROR:root:[KVP] Overwriting group best_test_params!


[Epoch 001][trn] cost  8.671412 (cla 8.6611, reg 0.0103), |grad| = 9.366352, acc = 0.01771 % (256.89sec)
           [tst] cost  8.598345, acc = 0.01451 %


ERROR:root:[KVP] Overwriting group best_train_params!
ERROR:root:[KVP] Overwriting group best_test_params!


[Epoch 002][trn] cost  8.552391 (cla 8.5402, reg 0.0122), |grad| = 0.550655, acc = 0.02380 % (255.76sec)
           [tst] cost  8.589051, acc = 0.01451 %


...

ERROR:root:[KVP] Overwriting group best_test_params!


[Epoch 008][trn] cost  8.450781 (cla 8.4255, reg 0.0253), |grad| = 3.718126, acc = 0.02032 % (255.86sec)
           [tst] cost  8.651540, acc = 0.04838 %
[Epoch 009][trn] cost  8.443307 (cla 8.4165, reg 0.0269), |grad| = 2.215768, acc = 0.03048 % (256.26sec)
           [tst] cost  8.508147, acc = 0.02661 %


ERROR:root:[KVP] Overwriting group best_test_params!


[Epoch 010][trn] cost  8.418008 (cla 8.3892, reg 0.0288), |grad| = 0.865371, acc = 0.04093 % (255.03sec)
           [tst] cost  8.515579, acc = 0.02903 %


ERROR:root:[KVP] Overwriting group best_train_params!


# Test phase
Now that the model is train it is enough to take the fwd function and apply it to new data.

In [ ]:
rewrite = True

out_path = './dumps/' 
kvs.load('best_test_params')

print "Saving output to: %s" % out_path

if not os.path.isdir(out_path) or rewrite==True:
    try:
        os.makedirs(out_path)
    except:
        pass
    
    a = []
    for i,d in enumerate(ds.test_iter()):
        fname = os.path.join(out_path, "%s" % ds.test_fnames[i])
        print fname,
        tmp = funcs['predict'](d[0], d[1])[0]
        a.append(np.mean(np.argmax(tmp, axis=1).flatten() == d[2].flatten()))
        scipy.io.savemat(fname, {'desc': tmp})
        print ", Acc: %7.5f %%" % (a[-1] * 100.0)
    print "\nAverage accuracy across all shapes: %7.5f %%" % (np.mean(a) * 100.0)
else:
    print "Model predictions already produced."

# Results

![](http://www.people.usi.ch/mascij/EG16_tutorial/shapenet_corr.png)